In [46]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn import preprocessing
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, plot_tree
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import roc_curve, roc_auc_score

In [47]:
vehicle = pd.read_csv("C:\Aalesh and Mandar\Datasets\Cases\Vehicle Silhouettes\Vehicle.csv")

In [48]:
vehicle.head()

,Comp,Circ,D.Circ,Rad.Ra,Pr.Axis.Ra,Max.L.Ra,Scat.Ra,Elong,Pr.Axis.Rect,Max.L.Rect,Sc.Var.Maxis,Sc.Var.maxis,Ra.Gyr,Skew.Maxis,Skew.maxis,Kurt.maxis,Kurt.Maxis,Holl.Ra,Class
0,95,48,83,178,72,10,162,42,20,159,176,379,184,70,6,16,187,197,van
1,91,41,84,141,57,9,149,45,19,143,170,330,158,72,9,14,189,199,van
2,104,50,106,209,66,10,207,32,23,158,223,635,220,73,14,9,188,196,saab
3,93,41,82,159,63,9,144,46,19,143,160,309,127,63,6,10,199,207,van
4,85,44,70,205,103,52,149,45,19,144,241,325,188,127,9,11,180,183,bus


In [49]:
x = vehicle.drop("Class", axis=1)
y = vehicle["Class"]

In [50]:
le = preprocessing.LabelEncoder()
le_y = le.fit_transform(y)
print(le.classes_)

['bus' 'opel' 'saab' 'van']


In [51]:
x_train,  x_test, y_train, y_test = train_test_split(x, le_y, test_size=0.3, random_state=2022)

In [52]:
lda = LinearDiscriminantAnalysis()
svm = SVC(kernel="linear", probability = True,  random_state = 2022)
dtc = DecisionTreeClassifier()
xgb = XGBClassifier()
stack = StackingClassifier([("lda", lda),("svm",svm),("dtc",dtc)],final_estimator=xgb, passthrough=True)

In [53]:
stack.fit(x_train, y_train)
y_pred = stack.predict(x_test)
y_pred_prob = stack.predict_proba(x_test)[:,1]

In [54]:
print(accuracy_score(y_test, y_pred))

0.8110236220472441


In [55]:
stack.get_params()

{'cv': None,
 'estimators': [('lda', LinearDiscriminantAnalysis()),
  ('svm', SVC(kernel='linear', probability=True, random_state=2022)),
  ('dtc', DecisionTreeClassifier())],
 'final_estimator__objective': 'binary:logistic',
 'final_estimator__use_label_encoder': None,
 'final_estimator__base_score': None,
 'final_estimator__booster': None,
 'final_estimator__callbacks': None,
 'final_estimator__colsample_bylevel': None,
 'final_estimator__colsample_bynode': None,
 'final_estimator__colsample_bytree': None,
 'final_estimator__early_stopping_rounds': None,
 'final_estimator__enable_categorical': False,
 'final_estimator__eval_metric': None,
 'final_estimator__feature_types': None,
 'final_estimator__gamma': None,
 'final_estimator__gpu_id': None,
 'final_estimator__grow_policy': None,
 'final_estimator__importance_type': None,
 'final_estimator__interaction_constraints': None,
 'final_estimator__learning_rate': None,
 'final_estimator__max_bin': None,
 'final_estimator__max_cat_thresho

In [56]:
kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 2022)
params = {"svm__C" : np.linspace(0.001, 8, 10),
          "dtc__max_depth" : [None, 4],
          "dtc__min_samples_leaf" : [1, 4],
          "dtc__min_samples_split" : [2, 4, 10],
          "final_estimator__n_estimators" : [50, 100],
          "final_estimator__learning_rate" : [0.1, 0.5],
          "final_estimator__max_depth" : [3, 5]}
          
         
gcv = GridSearchCV(stack, param_grid = params, verbose = 3, scoring = "neg_log_loss", cv = kfold)
gcv.fit(x, le_y)

Fitting 5 folds for each of 960 candidates, totalling 4800 fits
[CV 1/5] END dtc__max_depth=None, dtc__min_samples_leaf=1, dtc__min_samples_split=2, final_estimator__learning_rate=0.1, final_estimator__max_depth=3, final_estimator__n_estimators=50, svm__C=0.001;, score=-0.386 total time=   0.7s
[CV 2/5] END dtc__max_depth=None, dtc__min_samples_leaf=1, dtc__min_samples_split=2, final_estimator__learning_rate=0.1, final_estimator__max_depth=3, final_estimator__n_estimators=50, svm__C=0.001;, score=-0.387 total time=   0.6s
[CV 3/5] END dtc__max_depth=None, dtc__min_samples_leaf=1, dtc__min_samples_split=2, final_estimator__learning_rate=0.1, final_estimator__max_depth=3, final_estimator__n_estimators=50, svm__C=0.001;, score=-0.394 total time=   0.7s
[CV 4/5] END dtc__max_depth=None, dtc__min_samples_leaf=1, dtc__min_samples_split=2, final_estimator__learning_rate=0.1, final_estimator__max_depth=3, final_estimator__n_estimators=50, svm__C=0.001;, score=-0.463 total time=   0.6s
[CV 5/5]

KeyboardInterrupt: 

In [ ]:
print(gcv.best_params_)
print(gcv.best_score_)